# UPDATER OF THE CONFLICT LAYER IN STRATA

This script is used by Strata (https://strata.earthmap.org) in order to update the Conflict layers. It downloads data from the **ACLED** (https://acleddata.com/) using its API and stores it into GEE FeatureCollections. Then the FeatureCollections are read from GEE and an Image is generated using the location of the conflict data to buffer a square of 5000 meters around the point.



# **0** SET UP BEFORE RUNNING SCRIPTS

**0.1** Setup imports

In [ ]:
!pip install requests
!pip install python-dateutil
import requests
import ee
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from google.colab import userdata

### **0.2** Define Global variables

You will need to add these secrets:

1. **ACLED_API_KEY** ( these can be obtained here : https://developer.acleddata.com/ )
2. **ACLED_USER_EMAIL** (the email address used to obtain the API KEY)
2. **GEE_USER** ( your GEE user)
3. **ProjectID_GEE** ( the GEE-Google Cloud enabled project)

In [ ]:
# THE ACTUAL VALUES FOR API KEYS AND GEE PROJECT ID AND USERS ARE STORED AS SECRETS IN COLAB

# Set your Earth Engine project ID
project_id = userdata.get('ProjectID_GEE')

# Replace with your actual ACLED API key and email
ACLED_API_KEY = userdata.get('ACLED_API_KEY')  # Your unique API key from ACLED
ACLED_USER_EMAIL = userdata.get('ACLED_USER_EMAIL')  # The email you registered with at ACLED


# Set the prefix for GEE username and asset folder path
prefix = "users/" + userdata.get('GEE_USER')
assetFolder = f"{prefix}/ACLED/downloadedConflictData"  # Folderd path for storing ACLED data (each asset is a FeatureCollection)

**0.3** Initialize GEE connection

In [ ]:
ee.Authenticate()

# Initialize Earth Engine with your project ID
ee.Initialize(project=project_id)


**0.4** Define common methods for parts 1 and 2:
- Define Monitor for upload task


In [ ]:
import time

# Function to check task completion
def check_tasks_completion(task_ids, callbackFunction):
    """
    Continuously checks the status of a list of tasks until all are completed, failed, or cancelled.

    Args:
    task_ids (list): A list of task IDs to check.
    callbackFunction (function): A callback function to execute once all tasks are completed.

    """
    while True:
        all_done = True  # Flag to track if all tasks are done

        # Iterate over each task ID to check its status
        for task_id in task_ids:
            task = ee.data.getTaskStatus(task_id)[0]  # Get the status of the current task
            if task['state'] not in ['COMPLETED', 'FAILED', 'CANCELLED']:
                all_done = False  # If any task is not done, set flag to False
                break  # Exit the loop early if any task is not done

        if all_done:
            print("All tasks completed.")

            try:
                # Check if the callbackFunction is callable, then execute it
                if callable(callbackFunction):
                    print("Executing callback function.")
                    callbackFunction()
            except NameError:
                # Handle the case where the callbackFunction is not defined
                print("There is no callback declared.")

            break  # Exit the while loop if all tasks are done
        else:
            print("Tasks still running. Waiting...")
            time.sleep(30)  # Wait for 30 seconds before checking again

# Example usage
# check_tasks_completion(['task_id_1', 'task_id_2'], my_callback_function)


# **1** DOWNLOAD ACLED DATA TO THE GEE REPOSITORY

**1.1** Define methods necessary to connect to ACLED API

Define a method to download data from a date range

In [ ]:
import requests

def downloadData(start_date, end_date):
    """
    Download data from the ACLED API within the specified date range.

    Args:
    start_date (str): The start date for the data retrieval (format: 'YYYY-MM-DD').
    end_date (str): The end date for the data retrieval (format: 'YYYY-MM-DD').

    Returns:
    list: A list of dictionaries containing the retrieved data.
    """

    # Construct the base API request URL with necessary parameters
    base_url = (
        f'https://api.acleddata.com/acled/read?key={ACLED_API_KEY}&email={ACLED_USER_EMAIL}'
        f'&event_date={start_date}|{end_date}&event_date_where=BETWEEN&limit=10000'
        '&event_type=Riots:OR:event_type=Battles:OR:event_type=Explosions/Remote%20violence'
        ':OR:event_type=Violence%20against%20civilians:OR:event_type=Protests'
    )

    # Initialize an empty list to hold all the retrieved data
    all_data = []
    page = 1  # Start from the first page of the API response

    while True:
        # Construct the URL for the current page
        url = f"{base_url}&page={page}"
        print(f"Fetching page {page}: {url}")

        # Make the API request
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response data
            data = response.json()
            if 'count' in data and data['count'] > 0:
                # Add the retrieved data to the all_data list
                all_data.extend(data['data'])
                # Increment the page number for the next iteration
                page += 1
            else:
                # No more data to fetch; exit the loop
                break
        else:
            # Handle errors in the API request
            raise ValueError("Error Fetching data", response.status_code)

    return all_data

# Example usage
# data = downloadData('2023-01-01', '2023-06-01')
# print(data)


Define method to download conflicts for a period



In [ ]:
def get_conflicts_period(startDate, endDate):
    """
    Retrieve conflict data for a specified period.

    Args:
    startDate (datetime): The start date for the period.
    endDate (datetime): The end date for the period.

    Returns:
    list: A list of dictionaries containing conflict data for the specified period.
    """

    # Format the start and end dates as strings in the format 'YYYY-MM-DD'
    formatted_start_date = startDate.strftime("%Y-%m-%d")
    formatted_end_date = endDate.strftime("%Y-%m-%d")

    # Download conflict data from the ACLED API for the specified date range
    data_period = downloadData(formatted_start_date, formatted_end_date)

    return data_period

# Example usage
# from datetime import datetime
# conflicts = get_conflicts_period(datetime(2023, 1, 1), datetime(2023, 6, 1))
# print(conflicts)


Define method to check if an asset exists in the GEE repository

In [ ]:
# Function to check if the asset exists
def check_asset_exists(asset_name):
    try:
        asset_info = ee.data.getInfo(asset_name)
        return asset_info is not None
    except ee.EEException:
        return False


**Define methods to generate Feature Collection in GEE with the ACLED data**

Define methods to create FeatureCollection from downloaded ACLED conflicts

In [ ]:
def generate_feature_collection_conflicts_with_data(data_downloaded):
    """
    Generate a FeatureCollection from a list of data items.

    Args:
    data_downloaded (list): A list of dictionaries, where each dictionary contains
                            'longitude', 'latitude', 'event_type', and 'timestamp' keys.

    Returns:
    ee.FeatureCollection: An Earth Engine FeatureCollection containing features with
                          point geometries and properties.
    """
    # Create a list to hold the Features
    features = []

    # Iterate over the items in your data
    for item in data_downloaded:
        # Create a point geometry using the longitude and latitude from the item
        point = ee.Geometry.Point([float(item['longitude']), float(item['latitude'])])

        # Define the properties for the feature using the event type and timestamp
        properties = {'event_type': item['event_type'], 'timestamp': item['timestamp']}
        # include all properties but lat and long
        # properties = {k: v for k, v in item.items() if k not in ['latitude', 'longitude']}
        # This code fails beacuse there is too much data, we just keep event_type and timestamp

        # Create a feature from the point geometry and properties
        feature = ee.Feature(point, properties)

        # Add the feature to the list
        features.append(feature)

    # Create a FeatureCollection from the list of features
    feature_collection = ee.FeatureCollection(features)
    return feature_collection

def generate_feature_collection_conflicts_period(startDate, endDate):
    """
    Generate a FeatureCollection for conflicts occurring within a specified period.

    Args:
    startDate (datetime): The start date for the period.
    endDate (datetime): The end date for the period.

    Returns:
    ee.FeatureCollection: An Earth Engine FeatureCollection containing features for
                          conflicts occurring within the specified period.
    """
    # Retrieve the conflict data for the specified period
    data_period = get_conflicts_period(startDate, endDate)

    # Generate a FeatureCollection using the conflict data
    return generate_feature_collection_conflicts_with_data(data_period)


**Upload to GEE - GOAL OF PART 1**

In [ ]:

task_id_array = []  # List to store task IDs for tracking

def get_asset_id(startDate, endDate):
  	return assetFolder + '/Events_' + startDate.strftime('%Y%m%d') + '_' + endDate.strftime('%Y%m%d')

def upload_conflict_asset(feature_collection, startDate, endDate):
    """
    Uploads a feature collection to Google Earth Engine as an asset.

    Args:
    feature_collection (ee.FeatureCollection): The feature collection to be uploaded.
    startDate (datetime): The start date of the data period.
    endDate (datetime): The end date of the data period.

    Returns:
    bool: True if the task is started successfully.
    """
    # Generate the asset ID using the folder path and date range
    assetId = get_asset_id( startDate, endDate )

    # Define the export parameters for the task
    task = ee.batch.Export.table.toAsset(
        collection=feature_collection,
        description='monthly_acleddata_' + startDate.strftime('%Y%m%d') + '_' + endDate.strftime('%Y%m%d'),
        assetId=assetId
    )

    # Start the export task
    task.start()

    # Append the task ID to the task_id_array for tracking
    task_id_array.append(task.id)

    return task.id  # Task started successfully

def upload_conflict_period_to_asset(startDate, endDate):
    """
    Generates and uploads the feature collection for a specified period to GEE.

    Args:
    startDate (datetime): The start date of the data period.
    endDate (datetime): The end date of the data period.

    Returns:
    bool: True if the task is started successfully.
    """
    # Generate the feature collection for the specified period
    period_feature_collection = generate_feature_collection_conflicts_period(startDate, endDate)

    # Upload the feature collection as an asset
    return upload_conflict_asset(period_feature_collection, startDate, endDate)

# Get the current date
current_date = datetime.now()

# Upload conflict data for each month from 2024 to 2024
for year in range(2024, 2025):
  for month in range(1, 13):
      first_day = datetime(year, month, 1)  # First day of the month

      # Calculate the last day of the month
      if month == 12:
          # If it's December, set the last day of December
          last_day = datetime(year, month, 31)
      else:
          # Otherwise, calculate the last day normally
          last_day = datetime(year, month + 1, 1) - timedelta(days=1)

      # Check if the first day of the current iteration is beyond the current month
      if last_day > current_date.replace(day=1):
          print("Reached beyond the current month. Stopping the loop.")
          break

      asset_id = get_asset_id( first_day, last_day )

      if check_asset_exists(asset_id):
          print("The asset exists. No need to generate it again: " +asset_id )
      else:
          print("Generating asset for: " + asset_id)
          upload_conflict_period_to_asset(first_day, last_day)


check_tasks_completion(task_id_array, None)


# **2** GENERATE IMAGES FOR CONFLICT TYPES

**2.1** Generate a single ee.FeatureCollection that joins the data from 2023 with the monthly exports generated automaticallly

In [ ]:

# Load a FeatureCollection from a one-time download done back in January 2024
acled_downloaded_features = ee.FeatureCollection("users/openforisearthmap/ACLED/events_20220101_202312231")

# List all assets (FeatureCollections) inside the specified folder
# Adjust the folder path as necessary
asset_list = ee.data.listAssets(assetFolder)
print(asset_list)  # Print the list of assets to verify

# Iterate through the asset IDs and merge them with the original 2023 data
for asset in asset_list['assets']:
    print(f"{asset['id']} Merging with the other ACLED events")

    # Load the FeatureCollection for the current asset
    asset_fc = ee.FeatureCollection(asset['id'])

    # Merge the current FeatureCollection with the original 2023 data
    acled_downloaded_features = acled_downloaded_features.merge(asset_fc)

# Now, acled_downloaded_features contains the merged data from the original and all monthly updates



**2.2 Define methods to generate images per conflict type using the current GEE script**

Original at: https://code.earthengine.google.com/?scriptPath=users%2Fopenforisinitiative%2FEarthMap%3AStrata%2FPeace_and_Security%2FACLED

In [ ]:
# Define the list of all conflict types to be considered
CONFLICT_TYPES = ["Riots", "Battles", "Explosions/Remote violence", "Violence against civilians", "Protests"]

ALL_CONFLICTS_COMBINED = "AllConflicts"

# Function to get conflict features from the previous 12 months
# assuming we just downloaded data from the last month (if we are in March, then February 2024)
# we want to get the conflicts of the previous 12 months, meaning from 1st of March 2023 to end of February 2024
def get_conflict_features_last_12_months():
    # Get the current date and calculate the first day of the current month
    today = datetime.today()
    first_day_of_current_month = datetime(today.year, today.month, 1)

    # Calculate the last day of the previous month
    last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

    # Calculate the first day of the month, 12 months before the last day of the previous month
    first_day_12_months_before = (last_day_of_previous_month - relativedelta(months=12)).replace(day=1)

    # Convert the start and end dates to seconds since epoch
    start_month_seconds = ee.Date.fromYMD(first_day_12_months_before.year, first_day_12_months_before.month, first_day_12_months_before.day).millis().divide(1000)
    end_month_seconds = ee.Date.fromYMD(last_day_of_previous_month.year, last_day_of_previous_month.month, last_day_of_previous_month.day).millis().divide(1000)

    # Filter the downloaded features to get the conflicts within the specified time range
    return acled_downloaded_features.filter(ee.Filter.And(ee.Filter.gt("timestamp", start_month_seconds),
                                                          ee.Filter.lt("timestamp", end_month_seconds)))

# Function to get conflict features for a specific year
def get_conflict_features_in_year(year):
    # Convert the year to an Earth Engine number
    year = ee.Number(year)

    # Calculate the start and end of the year in seconds since epoch
    start_month_seconds = ee.Date.fromYMD(year, 1, 1).millis().divide(1000)
    end_month_seconds = ee.Date.fromYMD(year.add(1), 1, 1).millis().divide(1000)

    # Filter the downloaded features to get the conflicts within the specified year
    return acled_downloaded_features.filter(ee.Filter.And(ee.Filter.gt("timestamp", start_month_seconds),
                                                          ee.Filter.lt("timestamp", end_month_seconds)))

# Function to create an image from conflict features based on conflict type
def get_conflicts_image(conflicts, conflict_type):
    # Filter conflicts by conflict type if specified, otherwise use all conflict types
    if conflict_type and conflict_type != ALL_CONFLICTS_COMBINED:
        conflicts = ee.FeatureCollection(conflicts.filter(ee.Filter.inList("event_type", [conflict_type])))
    else:
        conflicts = ee.FeatureCollection(conflicts.filter(ee.Filter.inList("event_type", CONFLICT_TYPES)))

    # Buffer and bound the conflict features to create a conflict area
    conflicts = conflicts.map(lambda feature: feature.buffer(2500).bounds())  # this was generating a line! .buffer(2, 100).bounds(10)) changed to 100 everywhere

    # Rasterize the buffered points (polygons) into an image
    # Assign each feature a value of 1 for visualization
    type_band = conflicts.reduceToImage( ['timestamp'],ee.Reducer.first() ).rename('conflict');

    type_band = type_band.gt(0);

    # Return the self-masked image to only show areas with conflicts
    return type_band.selfMask()


In [ ]:
def export_image(conflict_type, region, fileName):
    """
    Export an image of conflicts for the last 12 months to Google Earth Engine.

    Args:
    conflict_type (str): The type of conflict to filter by. If None, use "AllConflicts".
    region (ee.Geometry): The region to export.
    fileName (str): The name of the file to export. If None, use the conflict type as the file name.

    """
    # Set the conflict type and file name
    conflict_type = conflict_type if conflict_type else ALL_CONFLICTS_COMBINED
    exportedFileName = fileName if fileName else conflict_type

    # Get the current date and calculate the first day of the current month
    today = datetime.today()
    first_day_of_current_month = datetime(today.year, today.month, 1)

    # Calculate the last day of the previous month
    last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

    # Calculate the first day of the month, 12 months before the last day of the previous month
    first_day_12_months_before = (last_day_of_previous_month - relativedelta(months=12)).replace(day=1)

    # Convert the start and end dates to seconds since epoch
    start_month_seconds = ee.Date.fromYMD(first_day_12_months_before.year, first_day_12_months_before.month, first_day_12_months_before.day).millis().divide(1000)
    end_month_seconds = ee.Date.fromYMD(last_day_of_previous_month.year, last_day_of_previous_month.month, last_day_of_previous_month.day).millis().divide(1000)

    # Filter the downloaded features to get the conflicts within the specified time range
    conflicts = acled_downloaded_features.filter(ee.Filter.And(ee.Filter.gt("timestamp", start_month_seconds),
                                                               ee.Filter.lt("timestamp", end_month_seconds)))

    # Get the conflict image for the last year
    image_last_year = get_conflicts_image(conflicts, conflict_type)

    # Set metadata for the image
    image_last_year = image_last_year.set("mosaicFrom", first_day_12_months_before.strftime('%Y%m%d'))
    image_last_year = image_last_year.set("mosaicTo", last_day_of_previous_month.strftime('%Y%m%d'))
    image_last_year = image_last_year.set("producedWithColab", "https://colab.research.google.com/drive/13eHZzmxk14SlEz_fK6Ah8igxgz28OoVd?usp=sharing")

    # Define the asset IDs for the export
    assetId = f"{prefix}/Strata_Global/PeaceAndSecurity/{exportedFileName}"
    assetId_temp = f"{assetId}_temp"

    # Check if the temp file exists and delete it if it does
    try:
        ee.data.getAsset(assetId_temp)
        ee.data.deleteAsset(assetId_temp)
        print(f"Deleted existing asset: {assetId_temp}")
    except ee.EEException as e:
        if "not found" in str(e):
            print(f"Asset does not exist: {assetId_temp}")
        else:
            raise

    image_last_year = image_last_year.reproject( 'EPSG:3857', None, 500 );

    # Export the image to an asset
    task = ee.batch.Export.image.toAsset(
        image=image_last_year,
        description=f"ACLED_{exportedFileName}",
        assetId=assetId_temp,
        pyramidingPolicy={'.default': 'max'},
        scale=1000,  # Scale of the image
        maxPixels=1e10,
        crs="EPSG:3857",  # Coordinate Reference System
        region=region
    )

    # Start the export task
    task.start()

    def callbackToCopyAsset():
        """
        Callback function to copy the temporary asset to the final destination and make it public.
        """
        print(f" Copying from {assetId_temp} to {assetId}")

        # Copy the asset to the final destination
        ee.data.copyAsset(assetId_temp, assetId, True)  # Overwrite if it already exists
        ee.data.deleteAsset(assetId_temp)
        print(f" Deleted {assetId_temp}")

        # Make the asset public
        acl = ee.data.getAssetAcl(assetId)
        acl['all_users_can_read'] = True
        ee.data.setAssetAcl(assetId, acl)
        print(f"Set {assetId} to be publicly accessible")

        print(f"{assetId} : Asset containing the conflicts for the last 12 months")

    check_tasks_completion([task.id], callbackToCopyAsset)


def generate_images(region):
    """
    Generate and export images for different conflict types.

    Args:
    region (ee.Geometry): The region to export.
    """
    # Export images for various conflict types
    export_image("Battles", region, None)
    export_image("Riots", region, None)
    export_image("Protests", region, None)
    export_image("Explosions/Remote violence", region, "ExplosionsRemoteViolence")
    export_image("Violence against civilians", region, "ViolenceAgainstCivilians")
    export_image(ALL_CONFLICTS_COMBINED, region, ALL_CONFLICTS_COMBINED )



**2.3** FINALLY : GENERATE ALL IMAGES GLOBALLY! GOAL OF PART 2



In [ ]:
# Define the region of interest
region = ee.Geometry.Rectangle([-180, -60, 180, 60], 'EPSG:4326', False)
testGeometry = ee.Geometry.Polygon([[[37.80477272160492, 50.12358491171723],[37.80477272160492, 48.35936521998567],[40.74910865910492, 48.35936521998567],[40.74910865910492, 50.12358491171723]]], 'EPSG:4326', False);
#region = ee.Geometry.Rectangle([-10, -10, 10, 10], 'EPSG:4326', False)

# Call the generate_images function with the specified region
generate_images(region)
